In [ ]:
# | default_exp _components.meta

In [ ]:
# | export

import builtins
import copy as cp
import functools
import inspect
import sys
import types
from functools import partial, wraps
from types import *
from typing import *

import docstring_parser
from mypy_extensions import DefaultNamedArg

In [ ]:
from aiokafka import AIOKafkaConsumer

# Fastcore replacement: patch & delegates

In [ ]:
# |exporti


def test_eq(a: Any, b: Any) -> None:
    "`test` that `a==b`"
    if a != b:
        raise ValueError(f"{a} != {b}")

## Patching

> copied from https://github.com/fastai/fastcore/blob/master/nbs/01_basics.ipynb

In [ ]:
# |exporti
F = TypeVar("F", bound=Callable[..., Any])


def copy_func(f: Union[F, FunctionType]) -> Union[F, FunctionType]:
    "Copy a non-builtin function (NB `copy.copy` does not work for this)"
    if not isinstance(f, FunctionType):
        return cp.copy(f)
    fn = FunctionType(
        f.__code__, f.__globals__, f.__name__, f.__defaults__, f.__closure__
    )
    fn.__kwdefaults__ = f.__kwdefaults__
    fn.__dict__.update(f.__dict__)
    fn.__annotations__.update(f.__annotations__)
    fn.__qualname__ = f.__qualname__
    fn.__doc__ = f.__doc__
    return fn

In [ ]:
def foo():
    """Test doc"""
    pass


a = cp.copy(foo)
b = cp.deepcopy(foo)

a.someattr = "hello"  # since a and b point at the same object, updating a will update b
test_eq(b.someattr, "hello")

assert a is foo and b is foo

However, with copy_func, you can retrieve a copy of a function without a reference to the original object:

In [ ]:
c = copy_func(foo)  # c is an indpendent object
assert c is not foo
assert c.__doc__ == """Test doc""", c.__doc__

In [ ]:
def g(x, *, y=3):
    return x + y


test_eq(copy_func(g)(4), 7)

In [ ]:
# |exporti


def patch_to(
    cls: Union[Type, Iterable[Type]], as_prop: bool = False, cls_method: bool = False
) -> Callable[[F], F]:
    "Decorator: add `f` to `cls`"
    if not isinstance(cls, (tuple, list)):
        cls = (cls,)  # type: ignore

    def _inner(f: F) -> F:
        for c_ in cls:
            nf = copy_func(f)
            nm = f.__name__
            # `functools.update_wrapper` when passing patched function to `Pipeline`, so we do it manually
            for o in functools.WRAPPER_ASSIGNMENTS:
                setattr(nf, o, getattr(f, o))
            nf.__qualname__ = f"{c_.__name__}.{nm}"
            if cls_method:
                setattr(c_, nm, MethodType(nf, c_))
            else:
                setattr(c_, nm, property(nf) if as_prop else nf)
        # Avoid clobbering existing functions
        # nosemgrep
        existing_func = globals().get(nm, builtins.__dict__.get(nm, None))
        return existing_func  # type: ignore

    return _inner

In [ ]:
class _T3(int):
    pass


@patch_to(_T3)
def foo(self):
    """Test doc"""
    pass


assert _T3.foo.__doc__ == """Test doc""", foo.__doc__

     
The @patch_to decorator allows you to monkey patch a function into a class as a method:

In [ ]:
class _T3(int):
    pass


@patch_to(_T3)
def func1(self, a):
    return self + a


t = _T3(1)  # we initilized `t` to a type int = 1
test_eq(t.func1(2), 3)  # we add 2 to `t`, so 2 + 1 = 3

     
You can access instance properties in the usual way via self:

In [ ]:
class _T4:
    def __init__(self, g):
        self.g = g


@patch_to(_T4)
def greet(self, x):
    return self.g + x


t = _T4("hello ")  # this sets self.g = 'helllo '
test_eq(
    t.greet("world"), "hello world"
)  # t.greet('world') will append 'world' to 'hello '

     
You can instead specify that the method should be a class method by setting cls_method=True:

In [ ]:
class _T5(int):
    attr = 3  # attr is a class attribute we will access in a later method


@patch_to(_T5, cls_method=True)
def func(cls, x):
    return cls.attr + x  # you can access class attributes in the normal way


test_eq(_T5.func(4), 7)

In [ ]:
# Additionally you can specify that the function you want to patch should be a class attribute with as_prop=True:


@patch_to(_T5, as_prop=True)
def add_ten(self):
    return self + 10


t = _T5(4)
test_eq(t.add_ten, 14)

     
Instead of passing one class to the @patch_to decorator, you can pass multiple classes in a tuple to simulteanously patch more than one class with the same method:

In [ ]:
class _T6(int):
    pass


class _T7(int):
    pass


@patch_to((_T6, _T7))
def func_mult(self, a):
    return self * a


t = _T6(2)
test_eq(t.func_mult(4), 8)
t = _T7(2)
test_eq(t.func_mult(4), 8)

In [ ]:
# | exporti


def eval_type(
    t: Sequence, glb: Optional[Dict[str, Any]], loc: Optional[Mapping[str, object]]
) -> Any:
    "`eval` a type or collection of types, if needed, for annotations in py3.10+"
    if isinstance(t, str):
        if "|" in t:
            return Union[eval_type(tuple(t.split("|")), glb, loc)]
        # nosemgrep
        return eval(t, glb, loc)  # nosec B307:blacklist
    if isinstance(t, (tuple, list)):
        return type(t)([eval_type(c, glb, loc) for c in t])
    return t


def union2tuple(t) -> Tuple[Any, ...]:  # type: ignore
    if getattr(t, "__origin__", None) is Union:
        return t.__args__  # type: ignore

    if sys.version_info >= (3, 10):
        if isinstance(t, UnionType):
            return t.__args__

    return t  # type: ignore


def get_annotations_ex(
    obj: Union[FunctionType, Type, F],
    *,
    globals: Optional[Dict[str, Any]] = None,
    locals: Optional[Dict[str, Any]] = None,
) -> Tuple[Dict[str, Any], Union[Any, Dict[str, Any], None], Dict[str, Any]]:
    "Backport of py3.10 `get_annotations` that returns globals/locals"
    if isinstance(obj, type):
        obj_dict = getattr(obj, "__dict__", None)
        if obj_dict and hasattr(obj_dict, "get"):
            ann = obj_dict.get("__annotations__", None)
            if isinstance(ann, types.GetSetDescriptorType):
                ann = None
        else:
            ann = None

        obj_globals = None
        module_name = getattr(obj, "__module__", None)
        if module_name:
            module = sys.modules.get(module_name, None)
            if module:
                obj_globals = getattr(module, "__dict__", None)
        obj_locals = dict(vars(obj))
        unwrap = obj
    elif isinstance(obj, types.ModuleType):
        ann = getattr(obj, "__annotations__", None)
        obj_globals = getattr(obj, "__dict__")
        obj_locals, unwrap = None, None
    elif callable(obj):
        ann = getattr(obj, "__annotations__", None)
        obj_globals = getattr(obj, "__globals__", None)
        obj_locals, unwrap = None, obj  # type: ignore
    else:
        raise TypeError(f"{obj!r} is not a module, class, or callable.")

    if ann is None:
        ann = {}
    if not isinstance(ann, dict):
        raise ValueError(f"{obj!r}.__annotations__ is neither a dict nor None")
    if not ann:
        ann = {}

    if unwrap is not None:
        while True:
            if hasattr(unwrap, "__wrapped__"):
                unwrap = unwrap.__wrapped__
                continue
            if isinstance(unwrap, functools.partial):
                unwrap = unwrap.func  # type: ignore
                continue
            break
        if hasattr(unwrap, "__globals__"):
            obj_globals = unwrap.__globals__

    if globals is None:
        globals = obj_globals
    if locals is None:
        locals = obj_locals

    return dict(ann), globals, locals  # type: ignore

In [ ]:
# | export


def patch(  # type: ignore
    f: Optional[F] = None, *, as_prop: bool = False, cls_method: bool = False
):
    "Decorator: add `f` to the first parameter's class (based on f's type annotations)"
    if f is None:
        return partial(patch, as_prop=as_prop, cls_method=cls_method)
    ann, glb, loc = get_annotations_ex(f)
    cls = union2tuple(
        eval_type(ann.pop("cls") if cls_method else next(iter(ann.values())), glb, loc)
    )
    return patch_to(cls, as_prop=as_prop, cls_method=cls_method)(f)

In [ ]:
class _T8(int):
    pass


@patch
def func(self: _T8, a):
    """Test doc"""
    return self + a


assert _T8.func.__doc__ == """Test doc""", func.__doc__

     
@patch is an alternative to @patch_to that allows you similarly monkey patch class(es) by using type annotations:

In [ ]:
class _T8(int):
    pass


@patch
def func(self: _T8, a):
    return self + a


t = _T8(1)  # we initilized `t` to a type int = 1
test_eq(t.func(3), 4)  # we add 3 to `t`, so 3 + 1 = 4
test_eq(t.func.__qualname__, "_T8.func")

Similarly to patch_to, you can supply a union of classes instead of a single class in your type annotations to patch multiple classes:

In [ ]:
class _T9(int):
    pass


@patch
def func2(x: Union[_T8, _T9], a):
    return x * a  # will patch both _T8 and _T9


t = _T8(2)
test_eq(t.func2(4), 8)
test_eq(t.func2.__qualname__, "_T8.func2")

t = _T9(2)
test_eq(t.func2(4), 8)
test_eq(t.func2.__qualname__, "_T9.func2")

     
Just like patch_to decorator you can use as_prop and cls_method parameters with patch decorator:

In [ ]:
@patch(as_prop=True)
def add_ten(self: _T5):
    return self + 10


t = _T5(4)
test_eq(t.add_ten, 14)

In [ ]:
class _T5(int):
    attr = 3  # attr is a class attribute we will access in a later method


@patch(cls_method=True)
def func(cls: _T5, x):
    return cls.attr + x  # you can access class attributes in the normal way


test_eq(_T5.func(4), 7)

In [ ]:
print("ok")

In [ ]:
def test_sig(f, b):
    "Test the signature of an object"
    if str(inspect.signature(f)) != b:
        raise ValueError(f"{inspect.signature(f)} != {b}")

# Fastcore meta deps

> Copied from https://github.com/fastai/fastcore/blob/master/nbs/07_meta.ipynb

In [ ]:
# |export


def _delegates_without_docs(
    to: Optional[F] = None,  # Delegatee
    keep: bool = False,  # Keep `kwargs` in decorated function?
    but: Optional[List[str]] = None,  # Exclude these parameters from signature
) -> Callable[[F], F]:
    "Decorator: replace `**kwargs` in signature with params from `to`"
    if but is None:
        but = []

    def _f(f: F) -> F:
        if to is None:
            to_f, from_f = f.__base__.__init__, f.__init__  # type: ignore
        else:
            to_f, from_f = to.__init__ if isinstance(to, type) else to, f  # type: ignore
        from_f = getattr(from_f, "__func__", from_f)
        to_f = getattr(to_f, "__func__", to_f)
        if hasattr(from_f, "__delwrap__"):
            return f
        sig = inspect.signature(from_f)
        sigd = dict(sig.parameters)
        if "kwargs" in sigd:
            k = sigd.pop("kwargs")
        else:
            k = None
        s2 = {
            k: v.replace(kind=inspect.Parameter.KEYWORD_ONLY)
            for k, v in inspect.signature(to_f).parameters.items()
            if v.default != inspect.Parameter.empty and k not in sigd and k not in but  # type: ignore
        }
        anno = {
            k: v
            for k, v in getattr(to_f, "__annotations__", {}).items()
            if k not in sigd and k not in but  # type: ignore
        }
        sigd.update(s2)
        if keep and k is not None:
            sigd["kwargs"] = k
        else:
            from_f.__delwrap__ = to_f
        from_f.__signature__ = sig.replace(parameters=list(sigd.values()))
        if hasattr(from_f, "__annotations__"):
            from_f.__annotations__.update(anno)
        return f

    return _f

A common Python idiom is to accept **kwargs in addition to named parameters that are passed onto other function calls. It is especially common to use **kwargs when you want to give the user an option to override default parameters of any functions or methods being called by the parent function.

For example, suppose we have have a function foo that passes arguments to baz like so:

In [ ]:
def baz(a, b: int = 2, c: int = 3) -> int:
    """Baz
    Params:
        a: something
        b: whatever
        c: whocares

    Returns:
        Nada
    """
    return a + b + c


def foo(c, a, **kwargs):
    return c + baz(a, **kwargs)


assert foo(c=1, a=1) == 7

The problem with this approach is the api for foo is obfuscated. Users cannot introspect what the valid arguments for **kwargs are without reading the source code. When a user tries tries to introspect the signature of foo, they are presented with this:

In [ ]:
inspect.signature(foo)

In [ ]:
inspect.signature(baz)

In [ ]:
print(baz.__doc__)

We can address this issue by using the decorator delegates to include parameters from other functions. For example, if we apply the delegates decorator to foo to include parameters from baz:

In [ ]:
@_delegates_without_docs(baz)
def foo(c, a, **kwargs):
    """Foo is great

    Params:
        c: c from foo
        a: a from foo
    """
    return c + baz(a, **kwargs)


test_sig(foo, "(c, a, *, b: int = 2)")
assert (
    foo.__doc__
    == "Foo is great\n\n    Params:\n        c: c from foo\n        a: a from foo\n    "
)
inspect.signature(foo)

In [ ]:
foo.__doc__

In [ ]:
# | export


def _format_args(xs: List[docstring_parser.DocstringParam]) -> str:
    return "\nArgs:\n - " + "\n - ".join(
        [f"{x.arg_name} ({x.type_name}): {x.description}" for x in xs]
    )


def combine_params(f: F, o: Union[Type, Callable[..., Any]]) -> F:
    """Combines docstring arguments of a function and another object or function

    Args:
        f: destination functions where combined arguments will end up
        o: source function from which arguments are taken from

    Returns:
        Function f with augumented docstring including arguments from both functions/objects
    """
    src_params = docstring_parser.parse_from_object(o).params
    #     logger.info(f"combine_params(): source:{_format_args(src_params)}")
    docs = docstring_parser.parse_from_object(f)
    #     logger.info(f"combine_params(): destination:{_format_args(docs.params)}")
    dst_params_names = [p.arg_name for p in docs.params]

    combined_params = docs.params + [
        x for x in src_params if not x.arg_name in dst_params_names
    ]
    #     logger.info(f"combine_params(): combined:{_format_args(combined_params)}")

    docs.meta = [
        x for x in docs.meta if not isinstance(x, docstring_parser.DocstringParam)
    ] + combined_params  # type: ignore

    f.__doc__ = docstring_parser.compose(
        docs, style=docstring_parser.DocstringStyle.GOOGLE
    )
    return f

In [ ]:
def f2(a: int = 0, b: str = "nada"):
    """
    Args:
        a: parameter a
        b: parameter bbbb
    """


def f1(b: str, c: int, **kwargs):
    """Function f1
    Args:
        b: parameter b
        c: parameter c

    Raises:
        ValueError: sometimes
    """


combine_params(f1, f2).__doc__

expected = """Function f1
Args:
    b: parameter b
    c: parameter c
    a: parameter a

Raises:
    ValueError: sometimes"""

assert f1.__doc__ == expected

In [ ]:
# | export


def delegates(
    o: Union[Type, Callable[..., Any]],
    keep: bool = False,
    but: Optional[List[str]] = None,
) -> Callable[[F], F]:
    """Delegates keyword agruments from o to the function the decorator is applied to

    Args:
        o: object (class or function) with default kwargs
        keep: Keep `kwargs` in decorated function?
        but: argument names not to include
    """

    def _inner(f: F, keep: bool = keep, but: Optional[List[str]] = but) -> F:
        def _combine_params(o: Union[Type, Callable[..., Any]]) -> Callable[[F], F]:
            def __combine_params(f: F, o: Union[Type, Callable[..., Any]] = o) -> F:
                return combine_params(f=f, o=o)

            return __combine_params

        @_combine_params(o)  # type: ignore
        @_delegates_without_docs(o, keep=keep, but=but)  # type: ignore
        @wraps(f)
        def _f(*args: Any, **kwargs: Any) -> Any:
            return f(*args, **kwargs)

        return _f

    return _inner

In [ ]:
def f2(a: str = "whatever", d: int = 42) -> None:
    """
    Args:
        a: parameter a
        b: parameter bbbb
    """
    pass


@delegates(f2)
def f1(b: str, c: int, **kwargs):
    """Function f1
    Args:
        b: parameter b
        c: parameter c

    Raises:
        ValueError: sometimes
    """
    pass


expected = """Function f1
Args:
    b: parameter b
    c: parameter c
    a: parameter a

Raises:
    ValueError: sometimes"""

assert f1.__doc__ == expected
assert len(inspect.signature(f2).parameters) == 2
assert len(inspect.signature(f1).parameters) == 4

In [ ]:
@delegates(f2)
def f3(b: str, c: int, **kwargs):
    """Function f1
    Args:
        b: parameter b
        c: parameter c

    Raises:
        ValueError: sometimes
    """
    pass


params = inspect.signature(f3).parameters
display(params)
assert len(params) == 4

In [ ]:
@delegates(AIOKafkaConsumer)
def f(a: int, **kwargs) -> str:
    """function a

    Args:
        a: parameter a

    Returns:
        things not stuff
    """
    print(f"{a=}")


assert len(f.__doc__) > 5000

print(f.__doc__)

In [ ]:
inspect.signature(f).parameters

We can optionally decide to keep **kwargs by setting keep=True:

In [ ]:
@delegates(baz, keep=True)
def foo(c, a, **kwargs):
    return c + baz(a, **kwargs)


inspect.signature(foo)

It is important to note that only parameters with default parameters are included. For example, in the below scenario only c, but NOT e and d are included in the signature of foo after applying delegates:

In [ ]:
def basefoo(e, d, c=2):
    pass


@delegates(basefoo)
def foo(a, b=1, **kwargs):
    pass


inspect.signature(
    foo
)  # e and d are not included b/c they don't have default parameters.

The reason that required arguments (i.e. those without default parameters) are automatically excluded is that you should be explicitly implementing required arguments into your function's signature rather than relying on delegates.

Additionally, you can exclude specific parameters from being included in the signature with the but parameter. In the example below, we exclude the parameter d:

In [ ]:
def basefoo(e, c=2, d=3):
    pass


@delegates(basefoo, but=["d"])
def foo(a, b=1, **kwargs):
    pass


test_sig(foo, "(a, b=1, *, c=2)")
inspect.signature(foo)

You can also use delegates between methods in a class. Here is an example of delegates with class methods:

In [ ]:
# example 1: class methods
class _T:
    @classmethod
    def foo(cls, a=1, b=2):
        pass

    @classmethod
    @delegates(foo)
    def bar(cls, c=3, **kwargs):
        pass


test_sig(_T.bar, "(c=3, *, a=1, b=2)")

Here is the same example with instance methods:

In [ ]:
# example 2: instance methods
class _T:
    def foo(self, a=1, b=2):
        pass

    @delegates(foo)
    def bar(self, c=3, **kwargs):
        pass


t = _T()
test_sig(t.bar, "(c=3, *, a=1, b=2)")

You can also delegate between classes. By default, the delegates decorator will delegate to the superclass:

In [ ]:
# class BaseFoo:
#     def __init__(self, e, c=2): pass

# @delegates()# since no argument was passsed here we delegate to the superclass
# class Foo(BaseFoo):
#     def __init__(self, a, b=1, **kwargs): super().__init__(**kwargs)

# test_sig(Foo, '(a, b=1, *, c=2)')

In [ ]:
# | export


def use_parameters_of(
    o: Union[Type, Callable[..., Any]], **kwargs: Dict[str, Any]
) -> Dict[str, Any]:
    """Restrict parameters passwed as keyword arguments to parameters from the signature of ``o``

    Args:
        o: object or callable which signature is used for restricting keyword arguments
        kwargs: keyword arguments

    Returns:
        restricted keyword arguments

    """
    allowed_keys = set(inspect.signature(o).parameters.keys())
    return {k: v for k, v in kwargs.items() if k in allowed_keys}

In [ ]:
assert use_parameters_of(AIOKafkaConsumer, api_version=0.1, radnom_param="random") == {
    "api_version": 0.1
}

In [ ]:
# | export


def filter_using_signature(f: Callable, **kwargs: Dict[str, Any]) -> Dict[str, Any]:
    """todo: write docs"""
    param_names = list(inspect.signature(f).parameters.keys())
    return {k: v for k, v in kwargs.items() if k in param_names}

In [ ]:
def f(a: int, *, b: str):
    pass


assert filter_using_signature(f, a=1, c=3) == {"a": 1}

In [ ]:
# | export

TorF = TypeVar("TorF", Type, Callable[..., Any])


def export(module_name: str) -> Callable[[TorF], TorF]:
    def _inner(o: TorF, module_name: str = module_name) -> TorF:
        o.__module__ = module_name
        return o

    return _inner

In [ ]:
@export("super.cool")
class A:
    pass


assert A.__module__ == "super.cool"